# # Overview: 
Load statistical metrics from EDF files and transform into a dataframe

Observe dataframe dimension and data types

Save dataframe to csv file

In [1]:
# Import the necessary packages from utils file
from utils import pd, np, mne

In [5]:
# Load one EDF file (PhysioNet uses .edf format)
path_name = '../data/raw/'
file_name = 'S001R04.edf'
raw = mne.io.read_raw_edf(path_name + file_name, preload=True)
raw.filter(1., 40.)  # Bandpass filtering

# Assume annotations are already present
events, event_id = mne.events_from_annotations(raw)

# Epoching (trial segmentation, e.g., 0–1s after event)
epochs = mne.Epochs(raw, events, event_id=event_id, tmin=0, tmax=1, baseline=None, preload=True)

# Extract trial IDs (sample index of each event)
trial_ids = epochs.events[:, 0]  # Sample index of event onset

# Create column names for features
feature_names = ['trial_id','subject_id']
for ch in range(1, 65):  # 64 channels
    feature_names.extend([
        f'ch{ch}_std',
        f'ch{ch}_mean',
        f'ch{ch}_max',
        f'ch{ch}_min'
    ])

# Create a table of features
features = []
labels = []

for trial_id, epoch, label in zip(trial_ids, epochs.get_data(), epochs.events[:, -1]):
    # epoch shape: (n_channels, n_times)
    channel_features = [trial_id, file_name]
    for channel in epoch:
        channel_features.extend([
            np.std(channel),
            np.mean(channel),
            np.max(channel),
            np.min(channel)
        ])
    features.append(channel_features)
    labels.append(label)

Extracting EDF parameters from /Users/miriamlandau/Documents/predict_hand_imagery/data/raw/S001R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 19999  =      0.000 ...   124.994 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 529 samples (3.306 s)

Used Annotations descriptions: [np.str_('T0'), np.str_('T1'), np.str_('T2')]
Not setting metadata
30 matching events found
No baseline correction applied
0 projection items activated
Using data 

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=1)]: Done  64 out of  64 | elapsed:    0.1s finished


In [6]:
# Transform dataset into pandas dataframe 
df = pd.DataFrame(features,columns=feature_names)
df['label'] = labels

In [9]:
# Analyze dataframe shape, columns, data types, and sample rows
print(df.info())
display(df.describe())
display(df.head())
print(f" dataframe shape {df.shape}")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Columns: 259 entries, trial_id to label
dtypes: float64(256), int64(2), object(1)
memory usage: 60.8+ KB
None


,trial_id,ch1_std,ch1_mean,ch1_max,ch1_min,ch2_std,ch2_mean,ch2_max,ch2_min,ch3_std,...,ch62_min,ch63_std,ch63_mean,ch63_max,ch63_min,ch64_std,ch64_mean,ch64_max,ch64_min,label
count,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,...,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000,3.000000e+01,30.000000,30.000000,30.000000
mean,9632.000000,0.000040,-5.384013e-07,0.000099,-0.000097,0.000043,-7.832657e-07,0.000103,-0.000102,0.000045,...,-0.000101,0.000045,7.744705e-07,0.000123,-0.000105,0.000042,7.138032e-07,0.000127,-0.000098,1.733333
std,5845.699628,0.000011,8.887085e-06,0.000031,0.000027,0.000011,9.368872e-06,0.000031,0.000025,0.000011,...,0.000025,0.000009,9.352656e-06,0.000034,0.000031,0.000006,6.938760e-06,0.000023,0.000021,0.827682
min,0.000000,0.000027,-2.107512e-05,0.000054,-0.000175,0.000029,-2.335466e-05,0.000056,-0.000179,0.000027,...,-0.000151,0.000033,-2.406747e-05,0.000057,-0.000164,0.000030,-1.407863e-05,0.000065,-0.000145,1.000000
25%,4820.000000,0.000033,-4.233780e-06,0.000074,-0.000111,0.000036,-5.949790e-06,0.000077,-0.000113,0.000038,...,-0.000123,0.000038,-4.782154e-06,0.000100,-0.000136,0.000038,-2.997388e-06,0.000114,-0.000111,1.000000
50%,9632.000000,0.000039,-1.635169e-06,0.000099,-0.000087,0.000040,-2.778214e-06,0.000100,-0.000094,0.000042,...,-0.000100,0.000043,2.071886e-06,0.000116,-0.000103,0.000042,-1.138494e-06,0.000127,-0.000099,1.500000
75%,14444.000000,0.000042,2.895811e-06,0.000108,-0.000075,0.000046,3.191965e-06,0.000117,-0.000085,0.000050,...,-0.000081,0.000053,4.806740e-06,0.000138,-0.000080,0.000044,2.658050e-06,0.000138,-0.000083,2.000000
max,19264.000000,0.000079,2.923329e-05,0.000169,-0.000060,0.000080,3.139056e-05,0.000175,-0.000065,0.000081,...,-0.000058,0.000066,2.461599e-05,0.000192,-0.000059,0.000054,2.113615e-05,0.000180,-0.000053,3.000000


,trial_id,subject_id,ch1_std,ch1_mean,ch1_max,ch1_min,ch2_std,ch2_mean,ch2_max,ch2_min,...,ch62_min,ch63_std,ch63_mean,ch63_max,ch63_min,ch64_std,ch64_mean,ch64_max,ch64_min,label
0,0,S001R04.edf,0.000040,-0.000021,0.000101,-0.000134,0.000038,-0.000023,0.000092,-0.000138,...,-0.000058,0.000038,1.294466e-05,0.000113,-0.000059,0.000038,-3.761820e-07,0.000114,-0.000081,1
1,672,S001R04.edf,0.000027,0.000005,0.000067,-0.000060,0.000032,0.000010,0.000075,-0.000081,...,-0.000069,0.000033,1.274179e-05,0.000100,-0.000079,0.000030,1.495558e-05,0.000130,-0.000053,3
2,1328,S001R04.edf,0.000051,0.000002,0.000093,-0.000122,0.000052,0.000002,0.000093,-0.000113,...,-0.000082,0.000033,-4.349705e-07,0.000080,-0.000080,0.000047,1.516147e-06,0.000112,-0.000106,1
3,2000,S001R04.edf,0.000038,0.000001,0.000095,-0.000088,0.000046,0.000003,0.000101,-0.000105,...,-0.000117,0.000048,9.954523e-06,0.000135,-0.000113,0.000046,8.908144e-06,0.000133,-0.000104,2
4,2656,S001R04.edf,0.000038,-0.000013,0.000097,-0.000105,0.000037,-0.000011,0.000103,-0.000088,...,-0.000080,0.000040,-1.790916e-05,0.000138,-0.000085,0.000039,-1.407863e-05,0.000152,-0.000079,1


 dataframe shape (30, 259)


In [10]:
#Save dataframe to csv in data folder
df.to_csv('../data/interim/eeg_motor_imagery.csv',index=False)


# Key Takeaways

**Dimension**

30 rows X 259 columns

Features are all float

**Column Naming Convention**

Features follow the pattern: `ch{channel_number}_{statistic}`

 Example: `ch1_std`, `ch1_mean`, `ch1_max`, `ch1_min`

**Key Identifiers**

`subject_id`: File name that uniquely identifies the trial subject

`trial_id`: Timestamp indicating when the trial occurred

**Note:**

 Dataset is shorter than wide, more trials should be added